## 필요한 라이브러리

In [2]:
import os
import io
import re
import numpy as np
import pandas as pd
import platform
from PIL import ImageFont, ImageDraw, Image
import matplotlib.pyplot as plt

import cv2
from google.cloud import vision

## Google OCR API 불러오기

In [3]:
# OCR API 불러오기
# \ => /로 변환해야 파일 인식 가능(안바꾸면 유니코드 에러 발생함)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'API 키 저장 위치'
 
client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

## 이미지 이진화
OCR인식률을 높이기 위함

cv_이미지명 형식으로 저장됨

In [4]:
img_path = 'test2.jpg'

In [7]:
img = cv2.imread(img_path)
img = cv2.resize(img, (0, 0), fx=0.2, fy=0.2)
# print(img.shape)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 최적의 gaussian 값을 찾아서 이진화 수행(block, cell 값 조정 필요함)
dst = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 7, 9)

# 이진화한 이미지 저장(cv_이미지명 형식으로 저장됨)
cv2.imwrite('cv_' + img_path, dst)

True

### 이미지 윤곽선 찾기
추가할 것) 윤곽선 찾은 것에서만 OCR 수행하기\n\n
위의 코드 이어서 수행하는 코드임!

In [ ]:
# 이미지에서 윤곽선 찾기
contours, _ = cv2.findContours(dst, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
# print(len(contours))
for pts in contours:
    if cv2.contourArea(pts) < 1000:
        continue
    
    approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)
    
    if len(approx) != 4:
        continue
    
    cv2.polylines(img, pts, True, (0, 0, 255))
    
cv2.imshow('src', img)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

### 이진화 전후 성능 비교

ex) 산도조절제를 인식하지 못했던 이전과 달리 이진화 후 인식이 가능해짐

이진화 전이 후보다 OCR 인식 시간이 오래걸림

In [21]:
img_origin = 'test2.jpg'
img_binary = 'cv_test2.jpg'

In [26]:
with io.open(img_origin, 'rb') as img_file:
    content = img_file.read()
    image = vision.Image(content=content)
 
    response = client.text_detection(image=image)
    texts = response.text_annotations

text = re.sub('\n', '', texts[0].description)

gredients_lst1 = []
special_char = '[^A-Za-z0-9가-힣]'
for first_split in text.split(','):
    material = [re.sub(special_char, ' ', f) for f in first_split.split()]
    for second_split in material:
        for third_split in second_split.split():
            gredients_lst1.append(third_split)

In [25]:
with io.open(img_binary, 'rb') as img_file:
    content = img_file.read()
    image = vision.Image(content=content)
 
    response = client.text_detection(image=image)
    texts = response.text_annotations

text = re.sub('\n', '', texts[0].description)

gredients_lst2 = []
special_char = '[^A-Za-z0-9가-힣]'
for first_split in text.split(','):
    material = [re.sub(special_char, ' ', f) for f in first_split.split()]
    for second_split in material:
        for third_split in second_split.split():
            gredients_lst2.append(third_split)

In [30]:
print(gredients_lst1, end=' ')
print()
print(gredients_lst2, end=' ')

['00OPEN빼빼로', '판매수익금롯데제과', 'Sweet', 'Home지역아동센터건립기금으로', '사용됩니다', '유통기한', '밑면표기일까지', '연', '월', '일', '식품유형', '초콜릿가공품', '내포장재질', '폴리프로필렌품목보고번호', 'F2', '양산', '19780614009498', 'F4', '대전', '19960242067376영업장의', '명칭', '롯데제과', '서울시', '영등포구', '양평로21길', '25및', '소재지원재료명', '준초콜릿', '설탕', '가공유지', '싱가포르산', '혼합분유', '외국산', '싱가포르', '프랑스', '네덜란드', '등', '전지분유', '코코아매스', '코코아매스', '유당', '밀가루', '밀', '미국산', '캐나다산', '준초콜릿II', '설탕', '혼합분유', '외국산', '싱가포르', '프랑스', '네덜란드', '등', '전지분유', '코코아매스', '가공유지', '싱가포르산', '코코아매스', '유당', '시드', '곡류가공품', '설탕', '쇼트닝', '가공연유', '가공버터', '전분가공품', '맥아엑기스', '전지분골드', '1', '정제소금', '산료', '효소제', '효모도조절제3종', '기타과당', '혼합제제', '합성향료', '밀가루', '비타민C', '카제인나트륨', '알긴산프로필렌글리콜', '혼합제제II', '바닐라추출물', '합성향밀', '대두', '우유', '땅콩', '함유', '직사광선', '및', '습기를', '피해', '서늘한', '곳에', '진열', '유통', '중', '변질품은', '구입상점', '및', '본사에서', '항상', '교환', '소비자기본법에', '의한', '피해', '보상', '휴지줍는', '고운마음', '안버리는', '밝은마음', '부정', '불량식품', '신고는', '국번없이', '1399', '달걀', '쇠고기', '돼지고기', '혼입', '가능', '고객지원센터080', '024', '6060수신자요금부담http', 'www'

## 이미지 불러오기

In [8]:
path = 'cv_test2.jpg'
# path = '../img2.jpg'

In [9]:
# 이미지 불러오기
with io.open(path, 'rb') as img_file:
    content = img_file.read()

## OCR 인식

OCR 수행 후 한글 문단 나누기로 인한 \n처리가 필요함

In [10]:
image = vision.Image(content=content)
 
response = client.text_detection(image=image)
texts = response.text_annotations

In [13]:
texts[0].description

'HE\n롯데지막ue\n국회는 전국민대방의\n식품유형 초콜릿가공품 내포장재질 플리프로필랜\n유통기한 밑면표기일까지(연.월.일)\nJF2(양산) 19780614009498,\n품목보고번호 F4(대전) 19960242067376\n영업장의 명칭 롯데제과(주) 서울시 영등포구 양평로21길 25\n및 소재지\n원재료명 준초콜릿 (설탕,가공유지(싱가포르산),혼합분유(외국산\n(싱가포르·프랑스-네덜란드 등); 전지분유, 코코아매스,코코아매\n스,유당), 밀가루(밀:미국산 캐나다산), 준초콜릿 (설탕, 혼합분유(외\n국산(싱가포르·프랑스·네덜란드 등 전지분유,코코아매스), 가공유\n지(싱가포르산) 코코아매스,유당), 시드(곡류가공품),설탕,쇼트닝,가\n공연유, 가공버터, 전분가공품, 맥아엑기스 전지분골드),정제소금, 산\n도조절제3종, 기타과당,혼합제제(합성향료, 밀가루, 비타민C,카체\n인나트륨, 알긴산프로필렌글리콜),혼합제제(바닐라추출물,합성향\n(료), 효소제,효모\n· 밀,대두,우유, 땅콩 함유\n-직사광선 및 습기를 피해 서늘한 곳에 진열, 유통 중 변질품은 구\n입상점 및 본사에서 항상 교환 ·소비자기본법에 의한 피해 보상\n• 휴지줍는 고운마음, 안버리는 밝은마음\n•부정, 불량식품 신고는 국번없이 1399\n•달걀,쇠고기 돼지고기 혼입 가능\n·유통 중 초콜릿 표면이 녹아 하얀 반점이 생길 수 있으나, 인체에\n\'무해하니 드셔도 괜찮습니다.\n영양정보\n나트륨 140mg7% 탄수화물 24g7% [당류 13g 13%\n지방 11g 20%\n트랜스지방 0g 포화지방 6g40%\n콜레스테롤 5mg미만 1%\n단백질 3g5%\n1일 영양성분 기준치에 대한 비율(%)은 2,000kcal 기준이므로 개인의 필요 열량에 따라 다꿀 수 있습니다..\n(2) 했다지요!\nSweet ECD\n6회 고객지원센터\n080-024-6060\n수신자요금부담\n"분리배출 해주세요\n총 내용량 39 g\n205 kcal\n편리한 분리배출\n주요\nA'

In [27]:
# print(texts[0].description)

### 오류 처리 코드

In [21]:
if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

# 제일 best인 전처리

## Text 전처리

In [11]:
# text = texts[0].description.replace("\n", "")

In [12]:
# text

'HE롯데지막ue국회는 전국민대방의식품유형 초콜릿가공품 내포장재질 플리프로필랜유통기한 밑면표기일까지(연.월.일)JF2(양산) 19780614009498,품목보고번호 F4(대전) 19960242067376영업장의 명칭 롯데제과(주) 서울시 영등포구 양평로21길 25및 소재지원재료명 준초콜릿 (설탕,가공유지(싱가포르산),혼합분유(외국산(싱가포르·프랑스-네덜란드 등); 전지분유, 코코아매스,코코아매스,유당), 밀가루(밀:미국산 캐나다산), 준초콜릿 (설탕, 혼합분유(외국산(싱가포르·프랑스·네덜란드 등 전지분유,코코아매스), 가공유지(싱가포르산) 코코아매스,유당), 시드(곡류가공품),설탕,쇼트닝,가공연유, 가공버터, 전분가공품, 맥아엑기스 전지분골드),정제소금, 산도조절제3종, 기타과당,혼합제제(합성향료, 밀가루, 비타민C,카체인나트륨, 알긴산프로필렌글리콜),혼합제제(바닐라추출물,합성향(료), 효소제,효모· 밀,대두,우유, 땅콩 함유-직사광선 및 습기를 피해 서늘한 곳에 진열, 유통 중 변질품은 구입상점 및 본사에서 항상 교환 ·소비자기본법에 의한 피해 보상• 휴지줍는 고운마음, 안버리는 밝은마음•부정, 불량식품 신고는 국번없이 1399•달걀,쇠고기 돼지고기 혼입 가능·유통 중 초콜릿 표면이 녹아 하얀 반점이 생길 수 있으나, 인체에\'무해하니 드셔도 괜찮습니다.영양정보나트륨 140mg7% 탄수화물 24g7% [당류 13g 13%지방 11g 20%트랜스지방 0g 포화지방 6g40%콜레스테롤 5mg미만 1%단백질 3g5%1일 영양성분 기준치에 대한 비율(%)은 2,000kcal 기준이므로 개인의 필요 열량에 따라 다꿀 수 있습니다..(2) 했다지요!Sweet ECD6회 고객지원센터080-024-6060수신자요금부담"분리배출 해주세요총 내용량 39 g205 kcal편리한 분리배출주요A'

In [14]:
# 대안2(제일 best)
text = re.sub('\n', '', texts[0].description)

In [15]:
gredients_lst = []
special_char = '[^A-Za-z0-9가-힣]'
for first_split in text.split(','):
    material = [re.sub(special_char, ' ', f) for f in first_split.split()]
    for second_split in material:
        for third_split in second_split.split():
            gredients_lst.append(third_split)

In [16]:
print(gredients_lst, end=' ')

['HE롯데지막ue국회는', '전국민대방의식품유형', '초콜릿가공품', '내포장재질', '플리프로필랜유통기한', '밑면표기일까지', '연', '월', '일', 'JF2', '양산', '19780614009498', '품목보고번호', 'F4', '대전', '19960242067376영업장의', '명칭', '롯데제과', '주', '서울시', '영등포구', '양평로21길', '25및', '소재지원재료명', '준초콜릿', '설탕', '가공유지', '싱가포르산', '혼합분유', '외국산', '싱가포르', '프랑스', '네덜란드', '등', '전지분유', '코코아매스', '코코아매스', '유당', '밀가루', '밀', '미국산', '캐나다산', '준초콜릿', '설탕', '혼합분유', '외국산', '싱가포르', '프랑스', '네덜란드', '등', '전지분유', '코코아매스', '가공유지', '싱가포르산', '코코아매스', '유당', '시드', '곡류가공품', '설탕', '쇼트닝', '가공연유', '가공버터', '전분가공품', '맥아엑기스', '전지분골드', '정제소금', '산도조절제3종', '기타과당', '혼합제제', '합성향료', '밀가루', '비타민C', '카체인나트륨', '알긴산프로필렌글리콜', '혼합제제', '바닐라추출물', '합성향', '료', '효소제', '효모', '밀', '대두', '우유', '땅콩', '함유', '직사광선', '및', '습기를', '피해', '서늘한', '곳에', '진열', '유통', '중', '변질품은', '구입상점', '및', '본사에서', '항상', '교환', '소비자기본법에', '의한', '피해', '보상', '휴지줍는', '고운마음', '안버리는', '밝은마음', '부정', '불량식품', '신고는', '국번없이', '1399', '달걀', '쇠고기', '돼지고기', '혼입', '가능', '유통', '중', '초콜릿', '표면이', '녹아', '하얀', '반점이', '생길', '수', '있으나', '인체에', '무해하니', '드셔도'

In [30]:
# text = re.sub(special_char, '/', text)

In [31]:
# text

In [28]:
# lst = []
# special_char = ''
# for split in text.split():
#     x = [re.sub('[^A-Za-z0-9가-힣]', ' ', t) for t in split.split(',')]
#     # print(x)
#     for j in x:
#         lst += j.split()

In [35]:
# lst

In [36]:
# print(gredients_lst, end=' ')

## 원재료명 list 엑셀 파일 읽어오기
=> excel 파일로 제공되는 파일 가져옴
==> json을 읽어오는 방법도 있음

In [17]:
materials = pd.read_excel('../materials_all.xlsx')

In [18]:
materials.head()

,Unnamed: 0,원재료,대표원재료명,원재료이명
0,0,Abiu열매,Abiu열매,NaN
1,1,Agathi꽃,Agathi꽃,NaN
2,2,Agathi잎,Agathi잎,NaN
3,3,Alder Buckthorn,Alder Buckthorn,NaN
4,4,Algarrobo Blanco,Algarrobo Blanco,NaN


In [19]:
food_materials = pd.DataFrame({'원재료' : list(set(gredients_lst))})
result = pd.merge(materials, food_materials
                   , how="inner", right_on="원재료", left_on="대표원재료명")

In [20]:
print(len(list(result["대표원재료명"])), list(result["대표원재료명"]))

26 ['대두', '땅콩', '밀', '밀가루', '연', '코코아매스', '맥아엑기스', '돼지고기', '우유', '전지분유', '은', '유당', '알긴산프로필렌글리콜', '효모', '단백질', '설탕', '쇼트닝', '가공유지', '초콜릿', '가공버터', '전지분골드', '준초콜릿', '곡류가공품', '초콜릿가공품', '효소제', '혼합제제']


### 문제 해결 필요 부분
<br> 은 => 문제 해결하기 </br>
<br> 합성향 료 => 문제 해결하기 </br>

### 여러가지 Try(좋지않은 결과)

In [141]:
lst = []
special_char = ''
for split in text.split():
    x = [re.sub('[^A-Za-z0-9가-힣]', ' ', t) for t in split.split(',')]
    # print(x)
    for j in x:
        lst += j.split()

In [142]:
print(lst, end=' ')

['00OPEN빼빼로', '판매수익금롯데제과', 'Sweet', 'Home지역아동센터건립기금으로', '사용됩니다', '유통기한', '밑면표기일까지', '연', '월', '일', '식품유형', '초콜릿가공품', '내포장재질', '폴리프로필렌품목보고번호', 'F2', '양산', '19780614009498', 'F4', '대전', '19960242067376영업장의', '명칭', '롯데제과', '서울시', '영등포구', '양평로21길', '25및', '소재지원재료명', '준초콜릿', '설탕', '가공유지', '싱가포르산', '혼합분유', '외국산', '싱가포르', '프랑스', '네덜란드', '등', '전지분유', '코코아매스', '코코아매스', '유당', '밀가루', '밀', '미국산', '캐나다산', '준초콜릿II', '설탕', '혼합분유', '외국산', '싱가포르', '프랑스', '네덜란드', '등', '전지분유', '코코아매스', '가공유지', '싱가포르산', '코코아매스', '유당', '시드', '곡류가공품', '설탕', '쇼트닝', '가공연유', '가공버터', '전분가공품', '맥아엑기스', '전지분골드', '1', '정제소금', '산료', '효소제', '효모도조절제3종', '기타과당', '혼합제제', '합성향료', '밀가루', '비타민C', '카제인나트륨', '알긴산프로필렌글리콜', '혼합제제II', '바닐라추출물', '합성향밀', '대두', '우유', '땅콩', '함유', '직사광선', '및', '습기를', '피해', '서늘한', '곳에', '진열', '유통', '중', '변질품은', '구입상점', '및', '본사에서', '항상', '교환', '소비자기본법에', '의한', '피해', '보상', '휴지줍는', '고운마음', '안버리는', '밝은마음', '부정', '불량식품', '신고는', '국번없이', '1399', '달걀', '쇠고기', '돼지고기', '혼입', '가능', '고객지원센터080', '024', '6060수신자요금부담http', 'www'

In [85]:
text

'00OPEN빼빼로 판매수익금롯데제과 Sweet Home지역아동센터건립기금으로 사용됩니다.유통기한 밑면표기일까지(연.월.일)식품유형 초콜릿가공품|내포장재질 폴리프로필렌품목보고번호 F2(양산) 19780614009498,F4(대전)19960242067376영업장의 명칭 롯데제과㈜ 서울시 영등포구 양평로21길 25및 소재지원재료명 준초콜릿 | (설탕, 가공유지(싱가포르산),혼합분유(외국산·(싱가포르·프랑스·네덜란드 등);전지분유,코코아매스),코코아매스,유당), 밀가루(밀:미국산,캐나다산),준초콜릿II(설탕,혼합분유(외국산(싱가포르 • 프랑스·네덜란드 등);전지분유,코코아매스),가공유지(싱가포르산),코코아매스,유당),시드(곡류가공품),설탕,쇼트닝,가공연유,가공버터,전분가공품,맥아엑기스,전지분골드 1,정제소금,산료), 효소제,효모도조절제3종, 기타과당,혼합제제(합성향료,밀가루,비타민C, 카제인나트륨,알긴산프로필렌글리콜),혼합제제II(바닐라추출물,합성향밀,대두,우유,땅콩 함유•직사광선 및 습기를 피해 서늘한 곳에 진열, 유통 중 변질품은 구입상점 및 본사에서 항상 교환 • 소비자기본법에 의한 피해 보상• 휴지줍는 고운마음, 안버리는 밝은마음• 부정, 불량식품 신고는 국번없이 1399•달걀,쇠고기,돼지고기 혼입 가능•●• 고객지원센터080-024-6060수신자요금부담http://www.lottecont.co.kr중 초콜릿 표면이 녹아 하얀 반점이 생길 수 있으나, 인체에무해하니 드셔도 괜찮습니다.영양정보지방 11g 20%나트륨 140 mg7% | 탄수화물 24g7% | 당류 13g 13%트랜스지방 0g포화지방 6g 40%콜레스테롤 5mg미만 1 %*1일 영양성분 기준치에 대한 비율(%)은 2,000kcal 기준이므로 개인의 필요 열량에 따라 다를 수 있습니다.1 롯데제과단백질 3g5%눌러서분리배출 해주세요총 내용량 39 g205 kcalSweet Eco롯데제과와 함께 환경을 지켜요△ 편리한 분리배출'

In [99]:
lst = [i for i in '준초콜릿, 설탕, 가공유지, 혼합분유, 전지분유, 코코아매스, 유당, 밀가루, 밀, 준초콜릿II, 시드, 곡류가공품, 쇼트닝, 가공연유, 가공버터, 전분가공품, 맥아엑기스, 전지분골드I, 정제소금, 산도조절제3종, 기타과당, 혼합제제I, 합성향료, 비타민C,  카제인나트륨, 알긴산프로필렌글리콜, 혼합제제II, 바닐라추출물, 효소제, 효모, 대두, 우유, 땅콩, 달걀, 쇠고기, 돼지고기'.split(', ')]

In [100]:
len(set(lst))

36

### 형태소 분석 수행 Try

한글에 대한 성능이 그닥 좋지 않음

In [ ]:
!pip install konlpy
import konlpy

x_data = []

for i, document in enumerate(texts[0].description):
    okt = konlpy.tag.Okt()
    clean_words = []
    for word in okt.pos(document, stem=True): #어간 추출
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']: #조사, 어미, 구두점 제외 
            clean_words.append(word[0])
    print(clean_words) #['스토리', '진짜', '너무', '노잼']
    document = ' '.join(clean_words)
    print(document) #스토리 진짜 너무 노잼
    x_data[i] = document